In [1]:
import sys
import numpy as np 
import os
import time
from PIL import Image
import matplotlib.pylab as plt1
import matplotlib.pyplot as plt
import cv2
from datetime import datetime
from pynq import Xlnk
from pynq import Overlay
import DACSDC

##Please enter you team name here; set the sleep time for function send, it is 0 by default
teamname = 'AccelDrone'
interval_time= 0
batchsize = 500
##create default work dir
[DAC,imgdir,overlaydir,result,timedir,coordir,xmlpath,mycoord,myxml]=DACSDC.startup(teamname)
##mycoord: please keep your raw data here.
##myxml:  please keep your xml files here.

###end initializing###

sys.path.insert(0,'/home/xilinx/caffe-ssd')
sys.path.insert(0,'/home/xilinx/caffe-ssd/python')

#print(sys.path)
import caffe

In [5]:
DACSDC.reset()
totalimg = len(DACSDC.getnames())
sendlist = DACSDC.getpathbatch()

net_file= '/home/xilinx/jupyter_notebooks/DAC/MobileNetSSD_deploy.prototxt'  
caffe_model='/home/xilinx/jupyter_notebooks/DAC/MobileNetSSD_deploy.caffemodel'  
test_dir = "/home/xilinx/jupyter_notebooks/DAC/images"

if not os.path.exists(caffe_model):
    print("MobileNetSSD_deploy.affemodel does not exist,")
    print("use merge_bn.py to generate it.")
    exit()
net = caffe.Net(net_file,caffe_model,caffe.TEST)

CLASSES = ('background',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat', 'chair',
           'cow', 'diningtable', 'dog', 'horse',
           'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor')
tbatch = 0 

def preprocess(src):
    img = cv2.resize(src, (300,300))
    img = img - 127.5
    img = img * 0.007843
    return img

def postprocess(img, out):   
    h = img.shape[0]
    w = img.shape[1]
    box = out['detection_out'][0,0,:,3:7] * np.array([w, h, w, h])

    cls = out['detection_out'][0,0,:,1]
    conf = out['detection_out'][0,0,:,2]
    return (box.astype(np.int32), conf, cls)

def detect(imgfile):
    #print(imgfile)
    origimg = cv2.imread(imgfile)
    img = preprocess(origimg)
    
    img = img.astype(np.float32)
    img = img.transpose((2, 0, 1))

    net.blobs['data'].data[...] = img
    out = net.forward()  
    box, conf, cls = postprocess(origimg, out)

    for i in range(len(box)):
       p1 = (box[i][0], box[i][1])
       p2 = (box[i][2], box[i][3])
       cv2.rectangle(origimg, p1, p2, (0,255,0))
       p3 = (max(p1[0], 15), max(p1[1], 15))
       title = "%s:%.2f" % (CLASSES[int(cls[i])], conf[i])
       cv2.putText(origimg, title, p3, cv2.FONT_ITALIC, 0.6, (0, 255, 0), 1)
   
    return (origimg,box)

def t1(): 
   # for i in range (int(totalimg/batchsize)):
    for f in os.listdir(test_dir):
        global tbatch
        
        start = time.time()
        #print(test_dir + "/" + f)
        image,box=detect(test_dir + "/" + f)
        
        end = time.time()
        t= end -start
        #end timer for the batch
        
        #print (t)
        tbatch = tbatch + t 
#         print (tbatch)
        
      #  out = str (out_buffer)
#         print (out)
      #  f.append(out)
    return


t1()
print(tbatch)
DACSDC.write(tbatch, totalimg,teamname)


UnboundLocalError: local variable 'tbatch' referenced before assignment